In [ ]:
%cd /content/drive/MyDrive/개인 공부방

/content/drive/MyDrive/개인 공부방


In [ ]:
!git clone https://github.com/huggingface/trl.git
%cd trl/
!pip install .

fatal: destination path 'trl' already exists and is not an empty directory.
/content/drive/MyDrive/개인 공부방/trl
Processing /content/drive/MyDrive/개인 공부방/trl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd /content/drive/MyDrive/개인 공부방/DPO 실험재현

/content/drive/MyDrive/개인 공부방/DPO 실험재현


In [ ]:
!pip install git+https://github.com/huggingface/peft.git
!pip install bitsandbytes
!pip install deepspeed
!pip install wandb

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ac9p3dl1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ac9p3dl1
  Resolved https://github.com/huggingface/peft.git to commit 3ff90626b6c4ec5c611392298e0f0339132bcc24
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.6.3.dev0-py3-none-any.whl size=137745 sha256=fe2d4b0b3419f80ff8037ab06a572fbdf137f16885a551ea2c0ee92e5db3db26
  Stored in directory: /tmp/pip-ephem-wheel-cache-j1g69ohs/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 k

- GPU cuda 뜬다고 4bit 키면 아래 에러남 8bit도 동일
- 해당방법이 미세조정을 쓰는것으로 보임 (reward trainer 안에서) 그래서 사실상 4-bit, 8-bit로만 불러오면 안됨


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning

- 파라미터 세팅없이 기본으로 그냥 돌리면 A100에서도 cuda 터짐

CUDA out of memory. Tried to allocate 412.00 MiB. GPU 0 has a total capacty of 39.56 GiB of which 256.56 MiB is free. Process 31401 has 39.31 GiB memory in use. Of the allocated memory 35.97 GiB is allocated by PyTorch, and 1.79 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.

- 아래는 여러 에러를 해결한 버전으로

1. 학습 loss 이상현상 pad_token_id 및 padding_side 추가로 해결
```
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
tokenizer.padding_side = "right"
```

2. 4bit, 8bit 사용법
```
load_in_4bit or load_in_8bit = True
use_peft = True

및 내부 파라미터 여러개 변경함 해당 내용은 gptj_reward_modeling.py 참고
```



In [ ]:
%cd /content/drive/MyDrive/개인 공부방/DPO 실험재현
!python gptj_reward_modeling.py

/content/drive/MyDrive/개인 공부방/DPO 실험재현
2023-11-16 01:06:30.418807: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 01:06:30.418860: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 01:06:30.418892: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 01:06:32.586043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-11-16 01:06:34,235] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.p